In [37]:
import pandas as pd
import  os
import numpy as np
import psycopg2
import glob
import openpyxl

In [38]:
df = pd.read_csv("expense.csv")
df.head()

,Expense Date,Expense Description,Expense Account,Expense Account Code,Paid Through,Paid Through Account Code,Vendor,Branch Name,Project Name,Entry Number,...,Tax Amount,Expense Amount,Total,Reference#,Is Billable,Customer Name,Expense Reference ID,Recurrence Name,ExpenseReport Name,Is Reimbursable
0,2021-01-15,Exchange Gain or Loss,Other Expenses,223500005,MTN Mobile Money,111000003,NaN,Jetstream Africa,NaN,1,...,0.0,31.75,31.75,TID11003228620,False,NaN,2620362000000088608,NaN,NaN,False
1,2021-01-14,Exchange Gain or Loss,Other Expenses,223500005,MTN Mobile Money,111000003,NaN,Jetstream Africa,NaN,2,...,0.0,631.90,631.90,INV-000317,False,NaN,2620362000000088688,NaN,NaN,False
2,2021-01-14,Exchange Gain / Loss,Other Expenses,223500005,Petty Cash,111000002,NaN,Jetstream Africa,NaN,3,...,0.0,0.02,0.02,Eculine - UK,False,NaN,2620362000000090547,NaN,NaN,False
3,2021-07-09,FEE- EBANKING ACH OUTWARD: Tran ID-S237127,Bank Fees and Charges,223200001,Stanbic - GHS,111000007,NaN,Jetstream Africa,NaN,21,...,0.0,2.50,2.50,NaN,False,NaN,2620362000000135186,NaN,NaN,False
4,2021-07-19,FEE RTGS 000001750708 JETSTREAM AFRICA ECOBANK,Bank Fees and Charges,223200001,Stanbic - GHS,111000007,NaN,Jetstream Africa,NaN,22,...,0.0,30.00,30.00,NaN,False,NaN,2620362000000136073,NaN,NaN,False


In [39]:
df.columns = [x.lower().replace("#","number").replace(" ","_").replace("%","").replace(".","") for x in df.columns]
df.columns

Index(['expense_date', 'expense_description', 'expense_account',
       'expense_account_code', 'paid_through', 'paid_through_account_code',
       'vendor', 'branch_name', 'project_name', 'entry_number',
       'currency_code', 'exchange_rate', 'is_inclusive_tax', 'mileage_rate',
       'mileage_unit', 'distance', 'start_odometer_reading',
       'end_odometer_reading', 'mileage_type', 'claimant_email', 'tax_name',
       'tax_percentage', 'tax_type', 'tax_amount', 'expense_amount', 'total',
       'referencenumber', 'is_billable', 'customer_name',
       'expense_reference_id', 'recurrence_name', 'expensereport_name',
       'is_reimbursable'],
      dtype='object')

In [40]:
df.dtypes

expense_date                  object
expense_description           object
expense_account               object
expense_account_code           int64
paid_through                  object
paid_through_account_code      int64
vendor                        object
branch_name                   object
project_name                 float64
entry_number                   int64
currency_code                 object
exchange_rate                float64
is_inclusive_tax                bool
mileage_rate                 float64
mileage_unit                 float64
distance                     float64
start_odometer_reading       float64
end_odometer_reading         float64
mileage_type                  object
claimant_email               float64
tax_name                      object
tax_percentage               float64
tax_type                      object
tax_amount                   float64
expense_amount               float64
total                        float64
referencenumber               object
i

In [41]:
replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'int64' : 'float',
    'datetime64[ns]': 'varchar',
    'datetime64': 'timestamp',
    'bool' : 'varchar'
}
replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'float',
 'datetime64[ns]': 'varchar',
 'datetime64': 'timestamp',
 'bool': 'varchar'}

In [42]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'expense_date varchar, expense_description varchar, expense_account varchar, expense_account_code float, paid_through varchar, paid_through_account_code float, vendor varchar, branch_name varchar, project_name float, entry_number float, currency_code varchar, exchange_rate float, is_inclusive_tax varchar, mileage_rate float, mileage_unit float, distance float, start_odometer_reading float, end_odometer_reading float, mileage_type varchar, claimant_email float, tax_name varchar, tax_percentage float, tax_type varchar, tax_amount float, expense_amount float, total float, referencenumber varchar, is_billable varchar, customer_name float, expense_reference_id float, recurrence_name float, expensereport_name float, is_reimbursable varchar'

In [55]:
conn_string = "host= db.app.production.jetstreamafrica.com \
                dbname='jetstream-finance' \
                    user='ujetstreamfinance' password='Smehr5A7FGw4jEX@' "

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

print('opened database successfully')

opened database successfully


In [56]:
cursor.execute("drop table if exists expense;")

In [57]:
cursor.execute("create table expense \
    (expense_date date, expense_description varchar, expense_account varchar, expense_account_code varchar, paid_through varchar, paid_through_account_code varchar, vendor varchar, branch_name varchar, project_name float, entry_number float, currency_code varchar, exchange_rate float, is_inclusive_tax varchar, mileage_rate float, mileage_unit float, distance float, start_odometer_reading float, end_odometer_reading float, mileage_type varchar, claimant_email float, tax_name varchar, tax_percentage float, tax_type varchar, tax_amount float, expense_amount float, total float, referencenumber varchar, is_billable varchar, customer_name float, expense_reference_id varchar, recurrence_name float, expensereport_name float, is_reimbursable varchar)")

In [58]:
#insert values to tables
#save df to csv
df.to_csv('expense', header=df.columns, index=False, encoding='utf-8')
#open the csv file, save it as an object and upload to db
my_file = open("expense")
print('file opened in memory')

file opened in memory


In [59]:
SQL_STATEMENT = """
COPY expense FROM STDIN WITH
   CSV
   HEADER
   DELIMITER AS ','
"""
cursor.copy_expert(sql = SQL_STATEMENT, file=my_file)
print('file copied to db')

file copied to db


In [60]:
cursor.execute("grant select on table expense to public")
conn.commit()

cursor.close()
print('table expense made imported successfully')

table expense made imported successfully
